<a href="https://colab.research.google.com/github/PearlSikka/language-ninja/blob/master/Real_or_not%3F_NLP_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

In [ ]:
import os
import pathlib

# Upload the API token.
def get_kaggle():
  try:
    import kaggle
    return kaggle
  except OSError:
    pass

  token_file = pathlib.Path("~/.kaggle/kaggle.json").expanduser()
  token_file.parent.mkdir(exist_ok=True, parents=True)

  try:
    from google.colab import files
  except ImportError:
    raise ValueError("Could not find kaggle token.")

  uploaded = files.upload()
  token_content = uploaded.get('kaggle.json', None)
  if token_content:
    token_file.write_bytes(token_content)
    token_file.chmod(0o600)
  else:
    raise ValueError('Need a file named "kaggle.json"')
  
  import kaggle
  return kaggle


kaggle = get_kaggle()


In [ ]:
!kaggle competitions download -c nlp-getting-started

In [ ]:
df=pd.read_csv('/content/train.csv')
df.head()

In [ ]:
train_df=df.loc[:,['id','text','target']]
train_df.head()

In [ ]:
train_df.isnull().any()

In [ ]:
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords 
lem=WordNetLemmatizer()
def cleaned_text(text):
    split_word=text.split()
    cleaned_word=" ".join([lem.lemmatize(i) for i in split_word if i not in stopwords.words('english')
    and not i.startswith('@')])
    return cleaned_word

In [ ]:
train_df['text']=train_df['text'].apply(lambda x: cleaned_text(x))

In [ ]:
test_df=pd.read_csv('/content/test.csv')
test_df['text']=test_df['text'].apply(lambda x: cleaned_text(x))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500)

train_x=cv.fit_transform(train_df['text']).toarray()
train_y=train_df['target'].values

test_x=cv.transform(test_df['text']).toarray()

In [ ]:
train_df['target'].unique()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model=MultinomialNB()
model.fit(train_x,train_y)
pred_y=model.predict(test_x)

In [ ]:
result=pd.DataFrame()
result['id']=test_df['id']
result['target']=pred_y


In [ ]:
result.to_csv('result.csv',index=False)

In [ ]:
test_df.head()